In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model import MaskedAutoencoderViT
from torch.utils.data import DataLoader
import torch 
import lightning.pytorch as pl
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
!export CUDA_VISIBLE_DEVICES=0

from data import DummyDataset

In [3]:
model = MaskedAutoencoderViT(img_size=(65, 650), \
                                       patch_size=(5, 10), \
                                       in_chans=1)

dataset = DummyDataset()
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
# dataset = DummyDataset()
# # batch_size = 1
# dataloader = DataLoader(dataset)

# for batch in dataloader: 
#     inputs, targets = batch 
#     print(type(inputs))

<class 'torch.Tensor'>


In [ ]:
# %reload_ext tensorboard
# %tensorboard --logdir=lightning_logs/ --port 7000

In [ ]:
wandb.login()
logger = pl.loggers.WandbLogger(project="", name="", 
                                save_dir="/data0/practical-sose23/brain-age", log_model=False)

early_stop_callback = EarlyStopping(monitor="validation loss", min_delta=0.00, patience=3, verbose=False, mode="max")

trainer = pl.Trainer(callbacks=[early_stop_callback], overfit_batches=1.0, max_epochs=200, accelerator="gpu", logger=logger)
trainer.fit(model, dataloader_train, dataloader_val)
wandb.finish()

In [ ]:


trainer = pl.Trainer(limit_train_batches=100, max_epochs=100, devices=1)

trainer.fit(model=MaskedAutoencoderViT(img_size=(65, 650), \
                                       patch_size=(5, 10), \
                                       in_chans=1), \
            train_dataloaders=dataloader)

In [6]:
from typing import Iterable
import misc as misc
import lr_sched as lr_sched
def train_one_epoch(model: torch.nn.Module,
                    data_loader: Iterable, optimizer: torch.optim.Optimizer,
                    device: torch.device, epoch: int, loss_scaler,
                    log_writer=None,
                    args=None):
    model.train(True)
    metric_logger = misc.MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', misc.SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)
    print_freq = 20

    accum_iter = args.accum_iter

    optimizer.zero_grad()

    if log_writer is not None:
        print('log_dir: {}'.format(log_writer.log_dir))

    for data_iter_step, (samples, _) in enumerate(metric_logger.log_every(data_loader, print_freq, header)):

        # we use a per iteration (instead of per epoch) lr scheduler
        if data_iter_step % accum_iter == 0:
            lr_sched.adjust_learning_rate(optimizer, data_iter_step / len(data_loader) + epoch, args)

        samples = samples.to(device, non_blocking=True)

        with torch.cuda.amp.autocast():
            loss, _, _ = model(samples, mask_ratio=args.mask_ratio)

        loss_value = loss.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            sys.exit(1)

        loss /= accum_iter
        loss_scaler(loss, optimizer, parameters=model.parameters(),
                    update_grad=(data_iter_step + 1) % accum_iter == 0)
        if (data_iter_step + 1) % accum_iter == 0:
            optimizer.zero_grad()

        torch.cuda.synchronize()

        metric_logger.update(loss=loss_value)

        lr = optimizer.param_groups[0]["lr"]
        metric_logger.update(lr=lr)

        loss_value_reduce = misc.all_reduce_mean(loss_value)
        if log_writer is not None and (data_iter_step + 1) % accum_iter == 0:
            """ We use epoch_1000x as the x-axis in tensorboard.
            This calibrates different curves when batch size changes.
            """
            epoch_1000x = int((data_iter_step / len(data_loader) + epoch) * 1000)
            log_writer.add_scalar('train_loss', loss_value_reduce, epoch_1000x)
            log_writer.add_scalar('lr', lr, epoch_1000x)


    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    return {k: meter.global_avg for k, meter in metric_logger.meters.items()}

In [7]:
model = MaskedAutoencoderViT(img_size=(65, 650), \
                                       patch_size=(5, 10), \
                                       in_chans=1)

dataset = DummyDataset()
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
from misc import NativeScalerWithGradNormCount as NativeScaler
loss_scaler = NativeScaler
from torch.utils.tensorboard import SummaryWriter
log_writer = SummaryWriter(log_dir='log')
optimizer = torch.optim.Adam(model.parameters(), betas=(0.9, 0.95))
class myargs(): 
    def __init__(self): 
        self.mask_ratio = 0.75
        self.accum_iter = 1 
        self.warmup_epochs = 40
        self.lr = 0.01
args = myargs()
# print(args.accum_iter)
train_one_epoch(model, dataloader, optimizer, device, 1, loss_scaler, log_writer, args)

log_dir: log
torch.Size([1, 1, 65, 650])


RuntimeError: Input type (c10::Half) and bias type (float) should be the same

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print("Current device:", device)

In [ ]:
# import matplotlib.pyplot as plt
# x = model.pos_embed
# print(x.squeeze(0).shape)
# attn = torch.matmul(x.squeeze(0), x.squeeze(0).T)
# # attn[torch.eye(attn.shape[0], dtype=bool)] = 0

# # plt.plot(attn[0][torch.argsort(attn[0])])
# plt.imshow(attn)
# plt.show()

In [ ]:
# x = model.pos_embed
# for block in model.blocks:
#     x = block(x)

In [ ]:
# model(dummy_EEG_input)

In [ ]:
# device_count = torch.cuda.device_count()
# print(device_count)